# Retrieving and Writing New York Times article into a JSON file 
* Reference: https://medium.com/@danalindquist/using-new-york-times-api-and-jq-to-collect-news-data-a5f386c7237b

In [19]:
# !pip install requests

In [1]:
import requests

In [2]:
#need to get your own in New York Times API
api_key = 'MJOMDoBbNoULuAzFbbtAMXz4T0Ac0M7D'

In [3]:
#type in what to query in the movie review
query = 'election'

### NYT API: Movie review search
* New York Times API uses elastic search to perform its queries.
* New York Times separated out articles related to movies under "movie review"
* can only query for title, short_summary - no actual article body

In [173]:
# #get movie reviews by a certain person
# first_name = 'Miriam'
# last_name = 'Bale'
# name = first_name + '%20' + last_name
# url = 'https://api.nytimes.com/svc/movies/v2/critics/' + name + '.json'

In [174]:
# date_range = '2015-01-01:2023-01-01'

In [175]:
# url = 'https://api.nytimes.com/svc/movies/v2/reviews/search.json?query=' + query + '&api-key=' + api_key + '&opening-date='+ date_range

In [182]:
# query_params = {
#     "api-key": api_key,
#     "q": query,
#     "sort": "newest",
#     "fq": ["display_title", "summary_short"], #only search in title and summary
#     "page": 0,  # Pagination, page 0 will return the first page of results
#     "page_size": 100
# }

# response = requests.get(url, params=query_params)

# # Process the data as per your requirements
# if response.status_code == 200: #request code 200 means the call was successful
#     data = response.json()
#     articles = data['results']
#     for article in articles:
#         print(article['display_title'])
#         print(article)
#             #print("Title:", article[i]['display_title'])
#     #print(data)
# else:
#     print("Error:", response.status_code)

In [183]:
# # view data
# data

In [1]:
# data.keys()

In [2]:
# data['results'][1]

### NYT API: Article search (the only section we need to extract all titles)
* **RESTRICTION-** the maximum number of articles that can be returned at once: 10
* Use the Article Search API to look up articles by keyword. You can refine your search using filters and facets.
* reference: https://ashar180.medium.com/crawling-ny-times-api-for-relevant-articles-b6134b651054

In [4]:
import json
import requests
import time #for delaying retrieval

In [24]:
# url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=' + query + '&api-key=' + api_key

In [65]:
#need to get your own in New York Times API
api_key = 'MJOMDoBbNoULuAzFbbtAMXz4T0Ac0M7D'

In [67]:
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

In [ ]:
query_list = #movie list from Mike 

all_articles = []

for movie_title in query_list: 
    
    page_number = 0

    while True: 

        query_params = {
            "api-key": api_key,
            "q": query,
            "sort": "newest",
            "page": page_number  # Pagination, page 0 will return the first page of results
            #can set "fq": field("what to serach") to restrict fields to have specific values in the query 
        }

        response = requests.get(url, params=query_params)

        # Process the data as per your requirements
        if response.status_code == 200: #request code 200 means the call was successful
            data = response.json()
            total_hits = data['response']['meta']['hits']
            print(total_hits)

            articles = data['response']['docs']

            #check if there's any articles left to loop through
            if not articles: #if article returned is NOT empty aka if there's no articles (if empty or is zero, returns false)
                break #stop the loop 

            #loop to load article in json 
            for article in articles:
                # Extract relevant fields from the article
                article_info = {
                    "Headline": article.get('headline', {}).get('main', ''),
                    "Abstract": article.get('abstract', ''),
                    "Lead_Paragraph": article.get('lead_paragraph', ''),
                    "Snippet": article.get('snippet', ''),
                    "Published_date": article.get('pub_date', ''),
                    "Author": article.get('byline', {}).get('person', [{}])[0].get('firstname', '') + " " + article.get('byline', {}).get('person', [{}])[0].get('lastname', ''),
                    "News_desk": "news_desk": article.get('news_desk', ''),
                    "URL": article.get('web_url', ''),
                    "Source": article.get('source', '')
                }

                all_articles.append(article_info)

            page_number += 1

            #delay in abstracting bc there's a limit on how many articles you can retrieve based on time
            time.sleep(1)  

        elif response.status_code == 429:
            # Rate limit exceeded, retry after suggested duration (if provided) or with an increased delay
            retry_after = response.headers.get('Retry-After')
            if retry_after:
                time.sleep(int(retry_after))
            else:
                time.sleep(5)  # delay by 5 seconds if no retry duration is specified by the server 

        else:
            print("Error:", response.status_code)
            break

#save articles in a json file 
with open("nyt_articles.json", "w") as json_file:
    json.dump(all_articles, json_file, indent=4)

54858
54858
54858
54858
54858
54858
54858
54858
54858
54858
54858
54858
54858
54858


To view the data in the first page based on one title and not load in json:

In [76]:
#type in what to query in articles
query = 'mission impossible'

In [82]:
query_params = {
    "api-key": api_key,
    "q": query,
    "sort": "newest",
    "page": 0,  # Pagination, page 0 will return the first page of results
}

response = requests.get(url, params=query_params)

# Process the data as per your requirements
if response.status_code == 200: #request code 200 means the call was successful
    data = response.json()
    
    total_hits = data['response']['meta']['hits']
    print(total_hits)
    
    articles = data['response']['docs']
    for article in articles:
        
        print("-----")
        headline = article.get('headline', {}).get('main', '')
        snippet = article.get('snippet', '')
        web_url = article.get('web_url', '')

        if headline:
            print("Headline:", headline)

        if snippet:
            print("Snippet:", snippet)

        if web_url:
            print("URL:", web_url)

        #print(article['abstract'])
        #print("Author:", article['byline']['person'][0]['firstname'] + " " + article['byline']['person'][0]['lastname'])
        
        #print(article)
            #print("Title:", article[i]['display_title'])
    #print(data)
else:
    print("Error:", response.status_code)

21715
-----
Headline: Julian Barry, Who Made Lenny Bruce Into ‘Lenny,’ Dies at 92
Snippet: Mr. Barry’s scripts for a hit Broadway play and later a Hollywood movie about that rule-breaking comic helped give Mr. Bruce a lasting place in pop culture lore.
URL: https://www.nytimes.com/2023/07/27/movies/julian-barry-dead.html
-----
Headline: Judge Vacates Bowe Bergdahl’s Conviction and Dishonorable Discharge
Snippet: The Army colonel overseeing the court-martial trial had applied for a job with the Trump-era Justice Department, raising the appearance of bias.
URL: https://www.nytimes.com/2023/07/25/us/politics/bowe-bergdahl-conviction-overturned.html
-----
Headline: The Ongoing Mystery of Covid’s Origin
Snippet: We still don’t know how the pandemic started. Here's what we do know — and why it matters.
URL: https://www.nytimes.com/2023/07/25/magazine/covid-start.html
-----
Headline: Our Oppenheimer Moment: The Creation of A.I. Weapons
Snippet: Why the United States should lead the developmen

In [36]:
type(data)

dict

In [32]:
data['response']['docs'][1]

{'abstract': 'Accusations against the powerful sheriff in Clay County were there for anybody to find.',
 'web_url': 'https://www.nytimes.com/2023/07/29/pageoneplus/mississippi-open-secrets-records.html',
 'snippet': 'Accusations against the powerful sheriff in Clay County were there for anybody to find.',
 'lead_paragraph': 'The accusations of sexual misconduct against Eddie Scott, the sheriff of Clay County, Miss., will be new to almost everyone who reads our latest investigation.',
 'source': 'The New York Times',
 'multimedia': [{'rank': 0,
   'subtype': 'xlarge',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2023/07/30/multimedia/00insider-sheriff/00clay-sheriff-01-bptf-articleLarge.jpg',
   'height': 399,
   'width': 600,
   'legacy': {'xlarge': 'images/2023/07/30/multimedia/00insider-sheriff/00clay-sheriff-01-bptf-articleLarge.jpg',
    'xlargewidth': 600,
    'xlargeheight': 399},
   'subType': 'xlarge',
   'crop_name': 'articleLarge'},
  {'rank': 